# Outline:
## Attaining Relevant Data for BMSB data
1. ETL grab "City, Township, and Unorganized Territory in Minnesota" and "Brown Marmorated Stink Bug Survey Data, Minnesota" 
2. QAQC protocols:
    - check CRS (convert to 4326)
    - indenfity duplicate cities (sum their populations)
3. Create a 'Near Table' (calculates distance between cities)

In [2]:
import arcpy
import requests
import json
import pprint
import zipfile
from arcpy import env

#set working environment (where i want all my data to be saved to)
env.workspace = "D:\Spring_2022\AcGIS2\lab4\lab4_start1a\lab4_start1a.gdb"

### 1. ETL grab: 

In [3]:
def grab_MNgeo(desired_tag): 
    '''
    will return a list of files, which you must sift through to find location of desired zip file
    '''
    #intializing request
    base_url = "http://gisdata.mn.gov/api/3/action/package_search?q="
    package_information_url = base_url + desired_tag

    #requesting all information associated with tag from MN Geo Commons
    package_information = requests.get(package_information_url, auth = ('user', 'pass'), verify = False)

    #converting all the information to a json dictionary
    package_dict = json.loads(package_information.content)

    #diving into the json dictionary with the hopes of locating my desired data
    package_dict_result = package_dict["result"]['results']

    return package_dict_result

In [4]:
def extract_unzip(dictionary, location_idx, exit_location):
    '''
    Need to provide the 'local' of desired file from grab_MNgeo() ouput as a list with two number values (location_idx),
    the returned output of grab_MNgeo() (dictionary), 
    and the desired location you want to file as a string (exit_location).  
    '''
    ## Finally downloading the data
    r =requests.get(dictionary[location_idx[0]]['resources'][location_idx[1]]["url"])
    text = dictionary[location_idx[0]]['resources'][location_idx[1]]["url"]
    sliced = text.split("/")
    open(sliced[-1], 'wb').write(r.content)
    print(f"extracting {sliced[-1]}...")

    #unzipping and saving the data files to a folder within my current working directory
    with zipfile.ZipFile(sliced[-1], 'r') as zip_ref:
        zip_ref.extractall(exit_location)
    print('Done!')

C:\Users\runac\Downloads\Spring_2022\ArcGIS2\Alternative_pro_envs\arcgispro-py3-5572_lab1\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [5]:
def count_features_in_layers(featurefile):
    '''
    Useful function that will count number of rows in any given shapefile
    '''
    ## I now want to know how many points there are in my shapefile:
    lyrfile = featurefile
    result = arcpy.GetCount_management(lyrfile) #returns the number of rows in my shapefile
    print('{} has {} records'.format(lyrfile, result[0]))
    

### extracting 2 files from MNGeo
* using grab_MNgeo() and extract_unzip() 

In [6]:
#downloading and unzipping "Brown Marmorated Stink Bug Survey Data, Minnesota" data
MN_dict0 = grab_MNgeo("bmsb brown marmorated stink bug")
extract_unzip(MN_dict0, [0,0], r'D:\Spring_2022\AcGIS2\lab4\lab4_start1a\test_folder')

# #downloading and unzipping "City, Township, and Unorganized Territory in Minnesota" data
MN_dict1 = grab_MNgeo("civil township boundary")
extract_unzip(MN_dict1, [3,2], r'D:\Spring_2022\AcGIS2\lab4\lab4_start1a\test_folder')

C:\Users\runac\Downloads\Spring_2022\ArcGIS2\Alternative_pro_envs\arcgispro-py3-5572_lab1\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


extracting shp_biota_bmsb.zip...
Done!


C:\Users\runac\Downloads\Spring_2022\ArcGIS2\Alternative_pro_envs\arcgispro-py3-5572_lab1\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


extracting fgdb_bdry_mn_city_township_unorg.zip...
Done!


### 2. QAQC protocols (city/population data):
a.  Convert BMSB table to Point Layers
    * create a new column ("All") that sums "Adults" and "Nymphs" then create a "ground_truth" column based on "All"

b.  Editing "City, Township, and Unorganized Territory in Minnesota"
    * convert CRS to 4326
    * selecting only cities (excludes other territories and townships) and remove unnecessary fields/cols
    * indentify duplicate cities (sum their populations and then merge the poylgons)
    * convert city data to points (after Spatial Join!)
    
c. Spatial join of BMSB data to MN_Cities_WGS data (currently polygon) to assign BMSB to relevant/closet cities
    * appends a new column, "City_SJ", to BMSB 


## BMSB data

In [7]:
## Convert BMSB table to Point Layers
arcpy.management.XYTableToPoint("BMSBSurveyDataTable", 
                                r"D:\Spring_2022\AcGIS2\lab4\lab4_start1a\lab4_start1a.gdb\BMSBSurveyDataTable_Points", 
                                "Longitude", 
                                "Latitude", 
                                None, 
                                'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

#createing a new column that sums the results of "Adults" and "Nymphs"
arcpy.management.CalculateField("BMSBSurveyDataTable_Points", "ALL", "!Adults! + !Nymphs!", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

#creating a ground truth column based on the "ALL" and uses the following 'codeblock' to populate "G_Truth"
codeblock = """
def getClass(column):
    if int(column) == 0:
        return 0
    elif int(column) >= 1:
        return 1"""    

arcpy.management.CalculateField("BMSBSurveyDataTable_Points", "G_Truth", "getClass(!ALL!)", "PYTHON3", codeblock, "SHORT", "NO_ENFORCE_DOMAINS")

#delete unneeded columns within BMSB data (keep: "City;Year;CheckDate;Adults;Nymphs;ALL;G_Truth")
arcpy.management.DeleteField("BMSBSurveyDataTable_Points", "City;Year;CheckDate;Adults;Nymphs;ALL;G_Truth", "KEEP_FIELDS")

<Result 'BMSBSurveyDataTable_Points'>

## MN_Cities_data

In [20]:
#saves just the polygons for cities within MN
arcpy.conversion.FeatureClassToFeatureClass("SDW_GOVNT.CITY_TOWNSHIP_UNORG_TERR", "D:\Spring_2022\AcGIS2\lab4\lab4_start1a", "MN_Cities.shp", """ "CTU_CLASS" = 'CITY'""")

#converts "MN_Cities" spatial reference system to 4326
out_coordinate_system = arcpy.SpatialReference(4326)
arcpy.Project_management("MN_Cities", 'MN_Cities_WGS', out_coordinate_system)

# make a new field then copy the old field values into the new one (enables me to rename 'Feature_NA' to 'City_SJ')
#Note, named 'CITY_SJ' since you cannot spatial join (later) columns names when they are idenitcal
arcpy.management.CalculateField("MN_Cities_WGS", "CITY_SJ", "!FEATURE_NA!", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

#delete unnecessary columns
arcpy.management.DeleteField("MN_Cities_WGS", "CITY_SJ;POPULATION", "KEEP_FIELDS")

#removing duplicates cities by disolving the polygons that have the same 'CITY_SJ' 
#Simultaneously summing population for duplicate cities and udating the 'POPULATION' column
arcpy.management.Dissolve("MN_Cities_WGS", "MN_Cities_dis_WGS", 
                          "CITY_SJ", [["POPULATION", 'SUM']], 
                         "MULTI_PART", "DISSOLVE_LINES")

# make a new field then copy the old field values into the new one (enables me to rename 'Sum_Population' to 'POPULATION')
arcpy.management.CalculateField("MN_Cities_dis_WGS", "POPULATION", "!SUM_POPULATION!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

#delete unnecessary columns ('SUM_POPULA')
arcpy.management.DeleteField("MN_Cities_dis_WGS", "CITY_SJ;POPULATION", "KEEP_FIELDS")

#result: "MN_Cities_dis_WGS" as poylgon data

<Result 'MN_Cities_dis_WGS'>

In [22]:
#Let's check the results: if our duplicate cities successfully merged, should now have 853 Cities
count_features_in_layers("MN_Cities_WGS.shp") #still has duplicate cities...
count_features_in_layers("MN_Cities_dis_WGS") #success

MN_Cities_WGS.shp has 903 records
MN_Cities_dis_WGS has 853 records


## spatial join:
* ensuring that i append the new city designation to the table, appends the sum_population and new city labels (CITY_SJ) to the BMSB data 

In [23]:
#outputs a layer with MN_CITIES_dis_WGS columns (CITY_SJ, and Population) append to BMSB data
targetFeatures = "BMSBSurveyDataTable_Points"
joinFeatures = "MN_Cities_dis_WGS"
outfc =  r"D:\Spring_2022\AcGIS2\lab41\lab4_start1\BMSB_Pts_SJ"

# Create a new fieldmappings and add the two input feature classes. (this appends the columns from joinFeatures)
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(targetFeatures)
fieldmappings.addTable(joinFeatures)
 
#Run the Spatial Join tool, using the defaults for the join operation and join type
#used, "CLOSEST":joins features that is closest to a target feature via planar distance
arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "#", "#", fieldmappings, "CLOSEST_GEODESIC")

<Result 'D:\\Spring_2022\\AcGIS2\\lab41\\lab4_start1\\BMSB_Pts_SJ.shp'>

### Add 'BSMB_Sim[X]' fields to 'MN_Cities_dis_WGS' to record BMSB Presence (1) or Absence (0) in simulations

the 'BSMB_Sim[X]' fields will be populated during the Huff Model simulations (within the 'BMSB-Simulation' Notebook)

In [24]:
#convert each City to points (has been poylgons until now):
arcpy.management.FeatureToPoint("MN_Cities_dis_WGS", "MN_Cities_Pts_WGS.shp", "CENTROID")

#Note: error:  The length of Field Name must not be larger than 10 --an issue only for "BMSB_Sim100"
for i in range(0,100):
    fieldname = "BMSB_Sim"+str(i)
    arcpy.management.AddField("MN_Cities_Pts_WGS", fieldname, "SHORT", 1, None, None, '', "NULLABLE", "NON_REQUIRED", '')

#adding accuracy/confusion matrix columns to sum for each cities 100 simulations
col_list = ['overall_TP', 'overall_FP', 'overall_FN', 'overall_TN']
for name in col_list:
    arcpy.management.AddField("MN_Cities_Pts_WGS", name, "SHORT", 3, None, None, '', "NULLABLE", "NON_REQUIRED", '')
    
#creating a new column that calculates ranking each city based on its FP, and 
arcpy.management.AddField("MN_Cities_Pts_WGS", "Presence", "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
    
#creating a new column that calculates the Accuracy for each cities' predictions
arcpy.management.AddField("MN_Cities_Pts_WGS", "Accuracy", "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

#creating a new column that calculates ranking each city based on its FP, and 
arcpy.management.AddField("MN_Cities_Pts_WGS", "Rank", "SHORT", 3, None, None, '', "NULLABLE", "NON_REQUIRED", '')


<Result 'MN_Cities_Pts_WGS'>

### Generate Cities Near Table

This table returns a table with the distance of each city to every other city within MN. Duplicate city pairs have been deleted (city1 to city2 is the duplicate of city2 to city1). This table is nessecary for Huff model calulations (in 'BMSB-Simulation' Folder). 

In [25]:
#code makes a table for every city pair
arcpy.analysis.GenerateNearTable("MN_Cities_Pts_WGS", "MN_Cities_Pts_WGS", "MN_Cities_Pts_NearTable", None, "NO_LOCATION", "NO_ANGLE", "ALL", 1000, "GEODESIC")

#delete column "NEAR_RANK"
arcpy.management.DeleteField("MN_Cities_Pts_NearTable", "NEAR_RANK", "DELETE_FIELDS")

<Result 'MN_Cities_Pts_NearTable'>

In [26]:
# Delete duplicate pairs of cities

#set fields and feature class
fields = ('IN_FID', 'NEAR_FID', 'NEAR_DIST')
fc = "MN_Cities_Pts_NearTable"

rowcount = 1
with arcpy.da.UpdateCursor(fc, fields, sql_clause=(None, 'ORDER BY NEAR_DIST ASC')) as cursor:
    for row in cursor:
        if (rowcount % 2) == 0:
            cursor.deleteRow()
        rowcount += 1
            
print("Duplicates removed")
print(arcpy.management.GetCount("MN_Cities_Pts_NearTable"))

Duplicates removed
363378


In [27]:
#city 1: adding the city name fields (easier to read the name of the city istead of its FID)
arcpy.management.JoinField("MN_Cities_Pts_NearTable", "IN_FID", "MN_Cities_Pts_WGS", "FID", "CITY_SJ")
arcpy.management.AlterField("MN_Cities_Pts_NearTable", "CITY_SJ", "CITY_1", "CITY_1", "TEXT", 512, "NULLABLE", "DO_NOT_CLEAR")
#city2
arcpy.management.JoinField("MN_Cities_Pts_NearTable", "NEAR_FID", "MN_Cities_Pts_WGS", "FID", "CITY_SJ")
arcpy.management.AlterField("MN_Cities_Pts_NearTable", "CITY_SJ", "CITY_2", "CITY_2", "TEXT", 512, "NULLABLE", "DO_NOT_CLEAR")

<Result 'MN_Cities_Pts_NearTable'>

In [ ]:
# adds probability column to near table
# ONLY RUN ONCE
fc = 'MN_Cities_Pts_NearTable'
arcpy.management.AddField(fc, "TransProb", "DOUBLE", None, None, None, "TransProb", "NULLABLE", "NON_REQUIRED", '')

In [34]:
#JOIN CITY_1 population to near table
arcpy.management.JoinField("MN_Cities_Pts_NearTable", "CITY_1", "MN_Cities_Pts_WGS", "CITY_SJ", "POPULATION")
arcpy.management.AlterField("MN_Cities_Pts_NearTable", "POPULATION", "CITY_1_POP", "CITY_1_POP", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")

#JOIN CITY_2 population to near table
arcpy.management.JoinField("MN_Cities_Pts_NearTable", "CITY_2", "MN_Cities_Pts_WGS", "CITY_SJ", "POPULATION")
arcpy.management.AlterField("MN_Cities_Pts_NearTable", "POPULATION", "CITY_2_POP", 'CITY_2_POP', "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")

<Result 'MN_Cities_Pts_NearTable'>